In [14]:
import os
import csv
import numpy as np
import sys
import datetime
import dateutil.parser
from datetime import datetime
import math
import random


np.set_printoptions(threshold=sys.maxsize)

In [15]:
def n_to_coords(num,n=31):
    #사각형의 번호로 몇 행 몇 열에 있는지 반환
    i = num//n
    j = num%n

    return i,j

def generation_square(squarecenter,n=31):
    #센터를 중심으로 가로,세로 1.5km의 정사각형을 n개의 정사각형으로 분할.
    l1=squarecenter[0]-0.007831555125996204*4 #약0.75km
    ln1=squarecenter[1]-0.009677410125732422*4#약0.75km
    l2=squarecenter[0]+0.007831555125996204*4 
    ln2=squarecenter[1]+0.009677410125732422*4
    square=[[l1,ln1],[l1,ln2],[l2,ln1],[l2,ln2]]
    set_of_unitsqaure=[]
    
    step_of_l=float((l2-l1)/n)
    step_of_ln=float((ln2-ln1)/n)
    for i in range(n):
        for j in range(n):
            temp=[[l1+i*step_of_l,ln1+j*step_of_ln],[l1+i*step_of_l,ln1+(j+1)*step_of_ln],[l1+(i+1)*step_of_l,ln1+(j+1)*step_of_ln],[l1+(i+1)*step_of_l,ln1+j*step_of_ln]]
            set_of_unitsqaure.append(temp)
    return set_of_unitsqaure

def fit_point_to_square(point,sofsquare):
    #점이 해당하는 사각형의 인덱스 반환
    for i in range(len(sofsquare)):
        if point[0] >= sofsquare[i][0][0] and point[0] <= sofsquare[i][2][0] and  point[1] >= sofsquare[i][0][1] and point[1] <= sofsquare[i][2][1]:
            return i


def generate_heatmap_by_point(point,sofsquare):
    dummy=[[0 for i in range(int(len(sofsquare)**(1/2)))]for j in range(int(len(sofsquare)**(1/2)))]
    #dummy = [[0]*int(math.sqrt(len(sofsquare)))]*int(math.sqrt(len(sofsquare)))
    q=fit_point_to_square(point,sofsquare)
    if q:
        i_,j_=  n_to_coords(q,int(math.sqrt(len(sofsquare))))
        emptiness=True
        x=0
        while emptiness:
            for i in range(i_-x,i_+x+1):
                for j in range(j_-x,j_+x+1):
                    if i >=0 and i< len(dummy) and  j >=0 and j< len(dummy):
                        dummy[i][j]=max(dummy[i][j],100/(math.log(x+1)+0.5))
                        #dummy[i][j]+=100/((x+1)*(x+1))
            x+=1
        
            can=[]
            for i in dummy:
                for j in i:
                    if j == 0:
                        can.append(j)
            if len(can)==0:
                emptiness = False
    return dummy
                    
def square_t_polygon(list,polygon):
    pol_dum=[]
    for j in list:    
        temp=[]
        for i in j:
            temp.append(str(i[0])+" "+str(i[1]))
        temp.append(temp[0])
        temp=",".join(temp)
        temp=polygon+"(("+temp+"))"
        pol_dum.append(temp)

    return pol_dum

def agg_heatmap(squarecenter,P):
    a= generation_square(squarecenter)
    can=[[0 for i in range(int(len(a)**(1/2)))]for j in range(int(len(a)**(1/2)))]
    #can=np.zeros([int(math.sqrt(len(a))),int(math.sqrt(len(a)))])
    for p in P:
        t=generate_heatmap_by_point(p,a)
        if len(t)>0:
            can=[[can[i][j]+t[i][j] for j in range(int(len(a)**(1/2)))]for i in range(int(len(a)**(1/2)))]
    return can,a

def CPVP_value(critical_point,can,square):
    
    a = can
    b= square #a=heatmap score, b= map
    c=critical_point
    f,p=n_to_coords(fit_point_to_square(c,b))
    m=np.max(a)
    if m == 0:
        val=0
    if m!=0:
        val=a[f][p]/m
    return val


def CPVP_int_1(criticalpoint, P):
    a,b=agg_heatmap(criticalpoint, P)
    c= CPVP_value(criticalpoint,a,b)
    return c



def collect_all(dir_path_="C:/Users/찬란한 돌멩이/Desktop/오리/현서"):
    dir_path = dir_path_
    collect=[]
    for (root, directories, files) in os.walk(dir_path):
        for file in files:
            file_path = os.path.join(root, file)
            a=[]
            with open(file_path, 'r') as f:
                    
                csvreader = csv.reader(f, delimiter=',')
                # 헤더(컬럼명) 건너뛰고 싶을 때

                for row in csvreader:
                    a.append(row)

            a=a[1]
            a=a[1:]
            t_=[]
            collect
            for i in a:
                if "T" in i:
                    if t_:
                        collect.append(t_)
                    t_=[]
                    t_.append(i)
                if "T" not in i:
                    t_.append(i)
        collect=np.array(collect)
        collect=collect[0:,0:3] 

    return collect

def collect_with_timewindow(collect,std_time,day_delta=5,sec_delta=120):
    dump=[]
    stand=dateutil.parser.isoparse(std_time)
    if len(collect)>0:
        for i in collect:
            a=dateutil.parser.isoparse(i[0])
            diff1 = stand-a
            diff2 = a-stand
            if diff1.days<=day_delta and diff1.days>0:
                if stand.second >= a.second and diff1.seconds<=sec_delta:
                    dump.append(i)
                if stand.second < a.second and diff2.seconds<=sec_delta:
                    dump.append(i)
    dump=np.array(dump)
    #print(dump)
    if len(dump)>0:
        dump=dump[0:,1:]
        #print(dump)
    dump=dump.astype(np.float64)
    return dump

def exp(datafile,day_delta=5,sec_delta=120,run=100):
    random.seed(1)
    import time
    import statistics
    cpvp_val_list=[]
    run_time_list=[]

    exp_code="exp_"+"D"+str(day_delta)+"S"+str(sec_delta)
    f = open("C:/Users/IME/Desktop/CPVP_실험결과/"+f"D{str(day_delta)}"+"/"+exp_code+".txt", 'w')
    f.write(exp_code+'\n')
    
    
    target_list_=[]
    target_list=datafile
    for i in range(run):
        a=random.choice(target_list)
        target_list_.append(a)
        
    exp_data=datafile
    for i in range(len(target_list_)):
        a=target_list_[i]
        std_time=a[0]
        std_coords=a[1:]
        std_coords=[float(i) for i in std_coords]

        cpvp_set=collect_with_timewindow(exp_data,std_time,day_delta,sec_delta)
        start=time.time()
        cpvp_val=CPVP_int_1(std_coords,cpvp_set)
        time_=time.time()-start

        cpvp_val_list.append(cpvp_val)
        run_time_list.append(time_)
        #print(f"{i}. "+f"std_time: {std_time}"+" / cpvp_val: "+str(cpvp_val)+f" / cpvp_run_time: {time_}")
        small_result=f"{i+1}. "+f"std_time: {std_time}"+" / cpvp_val: "+str(cpvp_val)+f" / cpvp_run_time: {time_}"+'\n'
        f.write(small_result)
    
    mean1 = statistics.mean(cpvp_val_list)
    mincpvp=min(cpvp_val_list)
    maxcpvp=max(cpvp_val_list)
    mean2 = statistics.mean(run_time_list)
    #print(f"cpvp_val_mean : {mean1}"+f" / cpvp_val_(max,min) : ({mincpvp},{maxcpvp})"+f" / cpvp_run_time_mean : {mean2}")
    f.write("result: "+f"cpvp_val_mean : {mean1}"+f" / cpvp_val_(max,min) : ({mincpvp},{maxcpvp})"+f" / cpvp_run_time_mean : {mean2}")
    f.write("")
    f.close()









In [16]:
a=collect_all("C:/Users/IME/Desktop/오리/현서")



In [8]:
for i in range(5):
    for j in range(5):
        exp(a,day_delta=i+1,sec_delta=60*(j+1),run=100)

In [18]:
for i in range(1):
     exp(a,day_delta=18,sec_delta=300,run=100)

In [20]:
for i in range(1):
    for j in range(5):
        exp(i,day_delta=i+1,sec_delta=60*(j+1),run=10,targetfile="C:/Users/IME/Desktop/오리/현서 실험",datafile="C:/Users/IME/Desktop/오리/현서")


KeyboardInterrupt: 

In [70]:
a='2023-05-02T01:10:07.773Z'
b='2023-05-01T02:10:09.773Z'
a=dateutil.parser.isoparse(a)
b=dateutil.parser.isoparse(b)
a.second
diff=a-b
diff.seconds


7

In [44]:
file_path="C:/Users/찬란한 돌멩이/Desktop/오리/현서/2023-04-27-1000 2_0427.csv"
a=[]
with open(file_path, 'r') as f:
        
    csvreader = csv.reader(f, delimiter=',')
    # 헤더(컬럼명) 건너뛰고 싶을 때

    for row in csvreader:
        a.append(row)

a=a[1]
a=a[1:]
t_=[]
collect=[]
for i in a:
    if "T" in i:
        if t_:
            collect.append(t_)
        t_=[]
        t_.append(i)
    if "T" not in i:
        t_.append(i)
collect=np.array(collect)
collect=collect[0:,0:3]
